In [1]:
# The code was removed by Watson Studio for sharing.

#### Install the necessary package.

In [2]:
!pip install watson-developer-cloud==1.5
!pip install --user ipython-sql 
!pip install --user ibm_db 
!pip install --user ibm_db_sa

#### Import the necessary libraries

In [3]:
import requests
import urllib.request
import time
import datetime
import csv
import re
import unicodedata
import ibm_db
from bs4 import BeautifulSoup
import pandas as pd 

#### I've pre-defined some redundant functions here so as to reduce the complexity and size of the code

In [4]:
#function to check if we have access to the website
def access_to_server(a):
    if(access.status_code == 200):
        print("Access to PlaneCrashInfo.com: Granted")
    else:
        print("Access to PlaneCrashInfo.com: Denied")
        
#function to pause for a little so that the website server does not flag us as a spammer
def take_a_break():
    time.sleep(0.1)      #we pause for 3 seconds

#function to access the website
def access_website(url):
    access = None
    access = requests.get(url,timeout=10)
    return access   

def get_data(search_string, regex):
    pattern = re.compile(regex)
    matches = pattern.finditer(search_string)

    for match in matches:
        match = match.span()

    span = search_string[match[0]:match[1]]
    return span

#predefine a dataframe to store out clean data
headers = ['Date','Time','Location','Operator','Flight#','Route','AC Type','Registration','cn/ln','Total Aboard','Passengers Aboard','Crew Aboard','Total Fatalities','Passenger Fatalities','Crew Fatalities','Ground','Summary']
df = pd.DataFrame(headers)
clean_data= df.T

#### Next, we set the url to the Plane Crash Info website and access the site with our requests library. We also set the base url to create a dictionary with years as the key.

In [5]:
base_url = "http://www.planecrashinfo.com"
access = requests.get('http://www.planecrashinfo.com/database.htm')
access_to_server(access)
source = BeautifulSoup(access.text, 'html.parser')

Access to PlaneCrashInfo.com: Granted


 #### Here we build a dictionary with the url stores as the value and the year as the key

In [6]:
a_tags = source.find_all("a")
years = {a.text.strip(): {"url": base_url + a["href"] if a["href"][0] == "/" else base_url + "/" + a["href"]} for a in a_tags if a.text.strip().isdigit()}

#### Here we pause for a little to avoid getting flagged as a spammer.

In [7]:
take_a_break()

#### Now we start collecting the unstructured data

In [14]:
done = 0
for year, database in years.items():       #first we loop for each year from 1920 until 2019
    print(year)
    #if year == '1925':
    #    break
    take_a_break()
    response_year = access_website(database["url"])
    access_to_server(response_year)
    parser_year = BeautifulSoup(response_year.content, 'html.parser')
    a_tags2 = parser_year.find_all("a")
    a_tags2 = [a for a in a_tags2 if "Return to Home" not in a.text]
    for a in a_tags2:
            # request to crash detail page
            response_crash = access_website(base_url + "/" + year + a["href"] if a["href"][0] == "/" else base_url + "/" + year + "/" + a["href"])
            if (response_crash.status_code == 404):
                continue
            parser_crash = BeautifulSoup(response_crash.content, 'html.parser')
            tr_tags = parser_crash.find_all("tr")
            tr_tags = tr_tags[1:]

            database = [tr.find_all("td")[1].text.strip() for tr in tr_tags]  
            x = database
            #Splitting the data in the aboard row
            aboard_row = x[9]
            #remove \xa0 from the string
            aboard_row = unicodedata.normalize("NFKD", aboard_row)
            #remove duplicate spaces from the string
            aboard_row = re.sub(' +', ' ',aboard_row)
            #split the string to extract the data easily
            split_aboard_row = aboard_row.split(" ")
            #find the total number of people aboard
            total_aboard = split_aboard_row[0]
            #find the total number of passengers aboard
            passengers_aboard = split_aboard_row[1]
            passengers_aboard = passengers_aboard[-1]
            #find the total number of crew members aboard
            crew_aboard = split_aboard_row[2]
            crew_aboard = crew_aboard[-2]

            #Splitting the data in the fatalities row
            fatalities_row = x[10]
            #remove \xa0 from the string
            fatalities_row = unicodedata.normalize("NFKD", fatalities_row)
            #remove duplicate spaces from the string
            fatalities_row = re.sub(' +', ' ',fatalities_row)
            #split the string to extract the data easily
            split_fatalities_row = fatalities_row.split(" ")
            #find the total number of fatalities
            total_fatalities = split_fatalities_row[0]
            #find the total number of passenger fatalities
            passenger_fatalities = split_fatalities_row[1]
            passenger_fatalities = passenger_fatalities[-1]
            #find the total number of crew fatalities
            crew_fatalities = split_fatalities_row[2]
            crew_fatalities = crew_fatalities[-2]

            #save data for reorganization
            ground = x[11]
            summary = x[12]
            x[9] = total_aboard
            x[10] = passengers_aboard
            x[11] = crew_aboard
            x[12] = total_fatalities

            u= [passenger_fatalities, crew_fatalities, ground, summary]
            x.extend(u)
            df1 = pd.DataFrame(x) 
            q= df1.T
            clean_data = clean_data.append(q)

1920
Access to PlaneCrashInfo.com: Granted
1921
Access to PlaneCrashInfo.com: Granted
1922
Access to PlaneCrashInfo.com: Granted
1923
Access to PlaneCrashInfo.com: Granted
1924
Access to PlaneCrashInfo.com: Granted
1925
Access to PlaneCrashInfo.com: Granted
1926
Access to PlaneCrashInfo.com: Granted
1927
Access to PlaneCrashInfo.com: Granted
1928
Access to PlaneCrashInfo.com: Granted
1929
Access to PlaneCrashInfo.com: Granted
1930
Access to PlaneCrashInfo.com: Granted
1931
Access to PlaneCrashInfo.com: Granted
1932
Access to PlaneCrashInfo.com: Granted
1933
Access to PlaneCrashInfo.com: Granted
1934
Access to PlaneCrashInfo.com: Granted
1935
Access to PlaneCrashInfo.com: Granted
1936
Access to PlaneCrashInfo.com: Granted
1937
Access to PlaneCrashInfo.com: Granted
1938
Access to PlaneCrashInfo.com: Granted
1939
Access to PlaneCrashInfo.com: Granted
1940
Access to PlaneCrashInfo.com: Granted
1941
Access to PlaneCrashInfo.com: Granted
1942
Access to PlaneCrashInfo.com: Granted
1943
Access

In [15]:
data_copy = clean_data
data_copy

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,Date,Time,Location,Operator,Flight#,Route,AC Type,Registration,cn/ln,Total Aboard,Passengers Aboard,Crew Aboard,Total Fatalities,Passenger Fatalities,Crew Fatalities,Ground,Summary
0,"September 17, 1908",1718,"Fort Myer, Virginia",Military - U.S. Army,?,Demonstration,Wright Flyer III,?,1,2,1,1,1,1,0,0,"During a demonstration flight, a U.S. Army fly..."
0,"September 07, 1909",?,"Juvisy-sur-Orge, France",?,?,Air show,Wright Byplane,SC1,?,1,0,1,1,0,0,0,Eugene Lefebvre was the first pilot to ever be...
0,"July 12, 1912",0630,"Atlantic City, New Jersey",Military - U.S. Navy,?,Test flight,Dirigible,?,?,5,0,5,5,0,5,0,First U.S. dirigible Akron exploded just offsh...
0,"August 06, 1913",?,"Victoria, British Columbia, Canada",Private,?,?,Curtiss seaplane,?,?,1,0,1,1,0,1,0,The first fatal airplane accident in Canada oc...
0,"September 09, 1913",1830,Over the North Sea,Military - German Navy,?,?,Zeppelin L-1 (airship),?,?,20,?,?,14,?,?,0,The airship flew into a thunderstorm and encou...
0,"October 17, 1913",1030,"Near Johannisthal, Germany",Military - German Navy,?,?,Zeppelin L-2 (airship),?,?,30,?,?,30,?,?,0,Hydrogen gas which was being vented was sucked...
0,"March 05, 1915",0100,"Tienen, Belgium",Military - German Navy,?,?,Zeppelin L-8 (airship),?,?,41,?,?,21,?,?,0,Crashed into trees while attempting to land af...
0,"September 03, 1915",1520,"Off Cuxhaven, Germany",Military - German Navy,?,?,Zeppelin L-10 (airship),?,?,19,?,?,19,?,?,0,"Exploded and burned near Neuwerk Island, when..."
0,"July 28, 1916",?,"Near Jambol, Bulgeria",Military - German Army,?,?,Schutte-Lanz S-L-10 (airship),?,?,20,?,?,20,?,?,0,"Crashed near the Black Sea, cause unknown."


In [16]:
#Save the dataframe as a CSV to Object Storage
#Commented out after saving once. 
#project.save_data(data=data_copy.to_csv(index=False),file_name='planecrashinfo_data1711.csv',overwrite=True)

{'file_name': 'planecrashinfo_data1711.csv',
 'message': 'File saved to project storage.',
 'bucket_name': 'crestview-donotdelete-pr-i9vprxww0objz7',
 'asset_id': '6859a3ce-b512-45f8-a851-8d5878697daf'}

#### Saving the dataframe into IBM db2 database as backup and to allow future access. 

In [18]:
#perform simplified database access using SQL "magic"
%load_ext sql
# %sql ibm_db_sa://my-username:my-password@dashdb-txn-sbox-yp-dal09-03.services.dal.bluemix.net:50000/BLUDB
#the cell after this one is hidden because it contains senstive information that provides access to my IBM Cloud account. The code is similar to the line above except for the username and password.

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [19]:
# The code was removed by Watson Studio for sharing.

'Connected: bfc60487@BLUDB'

In [17]:
data_copy.describe()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
count,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931,9931
unique,4539,1259,4091,2241,885,3798,2439,4660,3868,245,12,12,201,12,12,53,4808
top,"August 31, 1988",?,"Moscow, Russia",Aeroflot,?,?,Douglas DC-3,?,?,3,0,3,1,0,2,0,?
freq,8,3018,34,510,7300,1548,666,546,1336,556,2416,1954,766,2610,1822,9366,128
